In [1]:
import logging
import threading
import json
import itertools
#import modin.pandas as pd
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import ibm_db
import shap

from pandas_profiling import ProfileReport
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions.db import Database
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.enginelog import EngineLogging
from iotfunctions.dbtables import FileModelStore
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.dbtables import FileModelStore, DBModelStore
from mmfunctions.anomaly import (ARIMAForecaster, SpectralAnomalyScoreExt, KMeansAnomalyScore, 
                                 KMeansAnomalyScoreV2)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import MinCovDet
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [2]:
db_schema=None
class DatabaseDummy:
    tenant_id = '###_IBM_###'
    db_type = 'db2'
    model_store = FileModelStore('./')
    def _init(self):
        return

db = DatabaseDummy()
        
print (db.tenant_id)

2021-03-31T19:16:53.674 INFO iotfunctions.dbtables.__init__ Init FileModelStore with path: ./
###_IBM_###


In [3]:
# Get stuff in


df_input_raw = pd.read_csv('./BP_DATA/IoTData.csv', index_col=False, parse_dates=['DATETIME'], comment='#')

df_input_raw = df_input_raw.rename(columns={'DATETIME':'timestamp'}).\
                drop(columns=['UMIDADE_REL','KW','KW2'])
temp_air = sp.signal.resample(df_input_raw['TEMP_AIR'].astype(np.float64), 212840)

df_input_raw['entity'] = 'demand'
#df_input_raw['entity']=df_input_raw['DEVICE_ID']
#df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp').dropna()
df_input_raw['timestamp'] = pd.to_datetime(df_input_raw['timestamp'])
df_input = df_input_raw.set_index(['timestamp']).resample("T", label="right").ffill()

#df_input

df_input


2021-03-31T19:16:53.731 INFO numexpr.utils._init_num_threads Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-03-31T19:16:53.732 INFO numexpr.utils._init_num_threads NumExpr defaulting to 8 threads.


,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [4]:
df_input['TEMP_AIR'] = df_input['TEMP_AIR'].astype(np.float64)

In [5]:
df_input

,TEMP_AIR,entity
timestamp,,
2019-09-15 00:00:00,18.7,demand
2019-09-15 00:01:00,18.7,demand
2019-09-15 00:02:00,18.7,demand
2019-09-15 00:03:00,18.7,demand
2019-09-15 00:04:00,18.7,demand
...,...,...
2020-08-31 19:56:00,19.0,demand
2020-08-31 19:57:00,19.0,demand
2020-08-31 19:58:00,19.0,demand


In [6]:
ll = 5

df_large_list = []
#df_raw = df_input_raw.reset_index().set_index(['timestamp'])
for i in range(ll):
    df_new = df_input.copy()
    df_new['entity'] = 'entity' + str(i)
    df_large_list.append(df_new)

df_large = pd.concat(df_large_list)

In [7]:
df_large = df_large.reset_index().set_index(['entity','timestamp'])

In [8]:
df_large['TEMP_AIR'] = df_large['TEMP_AIR'].astype(np.float64)
df_large.index.levels[0].name

'entity'

In [9]:

jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}


EngineLogging.configure_console_logging(logging.DEBUG)

spsi = SpectralAnomalyScoreExt('TEMP_AIR', 12, 'Spectral', 'SpectralInv')
et = spsi._build_entity_type(columns = [Column('TEMP_AIR',Float())], **jobsettings)
spsi._entity_type = et
df_large = spsi.execute(df = df_large)


2021-03-31T19:16:55.965 DEBUG iotfunctions.enginelog.configure_console_logging Console logging has been configured. Level = 10
2021-03-31T19:16:55.967 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:55.968 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:55.968 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:55.969 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 8.3.1
2021-03-31T19:16:55.970 DEBUG iotfunctions.util.__init__ Starting trace
2021-03-31T19:16:55.971 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_SpectralAnomalyScoreExt_20210331171655
2021-03-31T19:16:55.972 DEBUG iotfunctions.util.__init__ auto_save None
2021-03-31T19:16:55.973 DEBUG iotfunctions.util.categorize_args categorizing arguments
2021-03-31T19:16:55.974 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
20

2021-03-31T19:16:57.952 DEBUG mmfunctions.anomaly._calc 50664112
2021-03-31T19:16:58.138 DEBUG mmfunctions.anomaly.score Spectral z-score max: 3041.351678326707,   Spectral inv z-score max: 3.142341132240664
2021-03-31T19:16:58.140 DEBUG mmfunctions.anomaly._calc ->
2021-03-31T19:16:58.148 DEBUG mmfunctions.anomaly._calc --->
2021-03-31T19:16:58.237 DEBUG mmfunctions.anomaly.execute Scoring done


In [10]:
db

In [11]:
jobsettings = { 'db': db, 
               '_db_schema': 'public', 'save_trace_to_file' : True}

k2mii = KMeansAnomalyScoreV2('TEMP_AIR', 3, True, 'KMeansV2')
et = k2mii._build_entity_type(columns = [Column('TEMP_AIR',Float())], **jobsettings)
k2mii._entity_type = et
df_large = k2mii.execute(df = df_large)

2021-03-31T19:16:59.682 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:59.684 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:59.686 DEBUG mmfunctions.anomaly.__init__ TEMP_AIR
2021-03-31T19:16:59.688 DEBUG iotfunctions.metadata.__init__ Initializing new entity type using iotfunctions 8.3.1
2021-03-31T19:16:59.689 DEBUG iotfunctions.util.__init__ Starting trace
2021-03-31T19:16:59.690 DEBUG iotfunctions.util.__init__ Trace name: auto_trace_test_entity_for_KMeansAnomalyScoreV2_20210331171659
2021-03-31T19:16:59.691 DEBUG iotfunctions.util.__init__ auto_save None
2021-03-31T19:16:59.692 DEBUG iotfunctions.util.categorize_args categorizing arguments
2021-03-31T19:16:59.694 WARNING iotfunctions.metadata.__init__ Created a logical entity type. It is not connected to a real database table, so it cannot perform any database operations.
2021-03-31T19:16:59.694 DEBUG iotfunctions.metadata.__init__ Initialized entity type 
LocalEntityType:TEST_ENTITY_FOR_KMEANSANOMALYS

'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.


2021-03-31T19:17:36.976 DEBUG numba.core.byteflow.__init__ bytecode dump:
>          0	NOP(arg=None, lineno=70)
           2	LOAD_GLOBAL(arg=0, lineno=70)
           4	LOAD_METHOD(arg=1, lineno=70)
           6	LOAD_FAST(arg=1, lineno=70)
           8	LOAD_FAST(arg=0, lineno=70)
          10	BINARY_SUBTRACT(arg=None, lineno=70)
          12	CALL_METHOD(arg=1, lineno=70)
          14	STORE_FAST(arg=2, lineno=70)
          16	LOAD_GLOBAL(arg=0, lineno=71)
          18	LOAD_METHOD(arg=2, lineno=71)
          20	LOAD_GLOBAL(arg=0, lineno=71)
          22	LOAD_ATTR(arg=3, lineno=71)
          24	LOAD_FAST(arg=2, lineno=71)
          26	LOAD_CONST(arg=1, lineno=71)
          28	LOAD_CONST(arg=2, lineno=71)
          30	CALL_FUNCTION_KW(arg=2, lineno=71)
          32	CALL_METHOD(arg=1, lineno=71)
          34	LOAD_METHOD(arg=4, lineno=71)
          36	CALL_METHOD(arg=0, lineno=71)
          38	RETURN_VALUE(arg=None, lineno=71)
2021-03-31T19:17:36.977 DEBUG numba.core.byteflow.run pending: deq

2021-03-31T19:17:37.151 DEBUG numba.core.ssa.reconstruct_ssa -----------------------------------BEFORE SSA-----------------------------------
2021-03-31T19:17:37.151 DEBUG numba.core.ssa.reconstruct_ssa label 0:
    X = arg(0, name=X)                       ['X']
    Y = arg(1, name=Y)                       ['Y']
    $2load_global.0 = global(np: <module 'numpy' from '/home/markus/.local/lib/python3.8/site-packages/numpy/__init__.py'>) ['$2load_global.0']
    $4load_method.1 = getattr(value=$2load_global.0, attr=square) ['$2load_global.0', '$4load_method.1']
    $10binary_subtract.4 = Y - X             ['$10binary_subtract.4', 'X', 'Y']
    $12call_method.5 = call $4load_method.1($10binary_subtract.4, func=$4load_method.1, args=[Var($10binary_subtract.4, stat_models.py:70)], kws=(), vararg=None) ['$10binary_subtract.4', '$12call_method.5', '$4load_method.1']
    euclidean_sq = $12call_method.5          ['$12call_method.5', 'euclidean_sq']
    $16load_global.6 = global(np: <module 'numpy'

2021-03-31T19:17:37.187 DEBUG numba.core.ssa.reconstruct_ssa ================================================================================
2021-03-31T19:17:37.214 DEBUG numba.core.byteflow.__init__ bytecode dump:
>          0	NOP(arg=None, lineno=1)
           2	LOAD_GLOBAL(arg=0, lineno=1)
           4	LOAD_FAST(arg=1, lineno=70)
           6	LOAD_FAST(arg=0, lineno=70)
           8	BINARY_SUBTRACT(arg=None, lineno=1)
          10	CALL_FUNCTION(arg=1, lineno=1)
          12	RETURN_VALUE(arg=None, lineno=1)
2021-03-31T19:17:37.215 DEBUG numba.core.byteflow.run pending: deque([State(pc_initial=0 nstack_initial=0)])
2021-03-31T19:17:37.216 DEBUG numba.core.byteflow.run stack: []
2021-03-31T19:17:37.217 DEBUG numba.core.byteflow.dispatch dispatch pc=0, inst=NOP(arg=None, lineno=1)
2021-03-31T19:17:37.218 DEBUG numba.core.byteflow.dispatch stack []
2021-03-31T19:17:37.219 DEBUG numba.core.byteflow.dispatch dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=1)
2021-03-31T19:17:37.220 DEBUG nu

...

2021-03-31T19:18:56.905 DEBUG mmfunctions.anomaly.score KMeans score max: 12.680183642841467
2021-03-31T19:18:56.906 DEBUG mmfunctions.anomaly._calc ->
2021-03-31T19:18:56.909 ERROR mmfunctions.anomaly._calc KMeansV2 score integration failed with name 'time_series_temperature' is not defined
2021-03-31T19:18:56.912 DEBUG mmfunctions.anomaly._calc --->
2021-03-31T19:18:57.085 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09-15 00:08:00', '2019-09-15 00:09:00',
               ...
               '2020-08-31 19:51:00', '2020-08-31 19:52:00',
               '2020-08-31 19:53:00', '2020-08-31 19:54:00',
               '2020-08-31 19:55:00', '2020-08-31 19:56:00',
               '2020-08-31 19:57:00', '2020-08-31 

'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.


2021-03-31T19:19:34.305 DEBUG mmfunctions.anomaly.score KMeans score max: 14.605699484087195
2021-03-31T19:19:34.306 DEBUG mmfunctions.anomaly._calc ->
2021-03-31T19:19:34.309 ERROR mmfunctions.anomaly._calc KMeansV2 score integration failed with name 'time_series_temperature' is not defined
2021-03-31T19:19:34.310 DEBUG mmfunctions.anomaly._calc --->
2021-03-31T19:19:34.382 DEBUG mmfunctions.anomaly._calc Timedelta:0 days 00:01:00 Index: DatetimeIndex(['2019-09-15 00:00:00', '2019-09-15 00:01:00',
               '2019-09-15 00:02:00', '2019-09-15 00:03:00',
               '2019-09-15 00:04:00', '2019-09-15 00:05:00',
               '2019-09-15 00:06:00', '2019-09-15 00:07:00',
               '2019-09-15 00:08:00', '2019-09-15 00:09:00',
               ...
               '2020-08-31 19:51:00', '2020-08-31 19:52:00',
               '2020-08-31 19:53:00', '2020-08-31 19:54:00',
               '2020-08-31 19:55:00', '2020-08-31 19:56:00',
               '2020-08-31 19:57:00', '2020-08-31 

'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.


2021-03-31T19:20:12.734 DEBUG mmfunctions.anomaly.score KMeans score max: 14.290815552387516
2021-03-31T19:20:12.737 DEBUG mmfunctions.anomaly._calc ->
2021-03-31T19:20:12.741 ERROR mmfunctions.anomaly._calc KMeansV2 score integration failed with name 'time_series_temperature' is not defined
2021-03-31T19:20:12.749 DEBUG mmfunctions.anomaly._calc --->
2021-03-31T19:20:12.869 DEBUG mmfunctions.anomaly.execute Scoring done


In [12]:
df_large.index

MultiIndex([('entity0', '2019-09-15 00:00:00'),
            ('entity0', '2019-09-15 00:01:00'),
            ('entity0', '2019-09-15 00:02:00'),
            ('entity0', '2019-09-15 00:03:00'),
            ('entity0', '2019-09-15 00:04:00'),
            ('entity0', '2019-09-15 00:05:00'),
            ('entity0', '2019-09-15 00:06:00'),
            ('entity0', '2019-09-15 00:07:00'),
            ('entity0', '2019-09-15 00:08:00'),
            ('entity0', '2019-09-15 00:09:00'),
            ...
            ('entity4', '2020-08-31 19:51:00'),
            ('entity4', '2020-08-31 19:52:00'),
            ('entity4', '2020-08-31 19:53:00'),
            ('entity4', '2020-08-31 19:54:00'),
            ('entity4', '2020-08-31 19:55:00'),
            ('entity4', '2020-08-31 19:56:00'),
            ('entity4', '2020-08-31 19:57:00'),
            ('entity4', '2020-08-31 19:58:00'),
            ('entity4', '2020-08-31 19:59:00'),
            ('entity4', '2020-08-31 20:00:00')],
           names=['enti

In [14]:
df_large.loc[['entity3']]

TEMP_AIR  Spectral  SpectralInv  KMeans
entity  timestamp                                                   
entity3 2019-09-15 00:00:00      18.7         0            0       0
        2019-09-15 00:01:00      18.7         0            0       0
        2019-09-15 00:02:00      18.7         0            0       0
        2019-09-15 00:03:00      18.7         0            0       0
        2019-09-15 00:04:00      18.7         0            0       0
...                               ...       ...          ...     ...
        2020-08-31 19:56:00      19.0         0            0       0
        2020-08-31 19:57:00      19.0         0            0       0
        2020-08-31 19:58:00      19.0         0            0       0
        2020-08-31 19:59:00      19.0         0            0       0
        2020-08-31 20:00:00      19.0         0            0       0

[506641 rows x 4 columns]